# Visualize alpha-dystroglycan and LAMP1 binding regions

In [ ]:
# Imports
import os
import pandas as pd
import polyclonal

In [ ]:
# this cell is tagged as `parameters` for papermill parameterization
func_scores = None

min_times_seen = None
n_selections = None

In [ ]:
# # Uncomment for running interactive
# func_scores = "../results/func_effects/averages/293T_entry_func_effects.csv"

# min_times_seen = 2
# n_selections = 8

In [ ]:
# Filter functional scores based on min times seen and selection number
functional_scores = pd.read_csv(func_scores)

# Add dummy phenotype column
functional_scores["phenotype"] = "functional_effect"

functional_scores

In [ ]:
# Plotting settings
alphabet = ['R','K','H','D','E','Q','N','S','T','Y','W','F','A','I','L','M','V','G','P','C','*']

addtl_tooltip_stats = [
    "n_selections",
    "times_seen",
]

addtl_slider_stats = {
    "times_seen" : 2,
    "n_selections" : 8,
}

## Heatmap of **alpha-dystroglycan** binding residues

In [ ]:
# Alpha dystroglycan binding sites
DG_sites = [
    120,
    121,
    125,
    150, # supported by 151 and 125 interactions
    151,
    256,
    257,
    258,
]

DG_chart = polyclonal.plot.lineplot_and_heatmap(
    data_df=functional_scores,
    stat_col="effect",
    category_col="phenotype",
    alphabet=alphabet,
    addtl_tooltip_stats=addtl_tooltip_stats,
    addtl_slider_stats=addtl_slider_stats,
    init_floor_at_zero=False,
    init_site_statistic="mean",
    show_zoombar=False,
    show_lineplot=False,
    sites=DG_sites,
)

DG_chart

## Heatmap of **LAMP1** binding residues

In [ ]:
# LAMP1 binding sites
LAMP1_sites = [
    92, # histidine triad
    93, # histidine triad
    172,
    173,
    188,
    192,
    195,
    197,
    198,
    200,
    201,
    202,
    204,
    206,
    207,
    211,
    216,
    230, # histidine triad
]

LAMP1_chart = polyclonal.plot.lineplot_and_heatmap(
    data_df=functional_scores,
    stat_col="effect",
    category_col="phenotype",
    alphabet=alphabet,
    addtl_tooltip_stats=addtl_tooltip_stats,
    addtl_slider_stats=addtl_slider_stats,
    init_floor_at_zero=False,
    init_site_statistic="mean",
    show_zoombar=False,
    show_lineplot=False,
    sites=LAMP1_sites,
)

LAMP1_chart